In [ ]:
import pandas as pd
import numpy as np

def calculate_optimal_pnl(df, heat_rate=10, mw=250, min_run_hours=3, min_down_hours=6):
    df['potential_pnl'] = (df['power_price'] - (df['gas_price'] * heat_rate)) * mw
    df['run_decision'] = False  # Indicates if the plant is running at a given hour
    df['pnl'] = 0.0  # Initialize PnL to 0

    n = len(df)

    def can_run(start_idx):
        """Determine if it's profitable to run starting from start_idx considering min run time."""
        if start_idx + min_run_hours > n:  # Not enough hours left to meet min run time
            return False, 0
        total_pnl = sum(df.iloc[start_idx:start_idx+min_run_hours]['potential_pnl'])
        return total_pnl > 0, total_pnl

    i = 0
    while i < n:
        profitable, total_pnl = can_run(i)
        if profitable:
            # Extend run time beyond min_run_hours as long as it's profitable
            extended_hours = 0
            for j in range(i + min_run_hours, min(i + min_run_hours + min_down_hours, n)):
                additional_pnl = df.iloc[j]['potential_pnl']
                if total_pnl + additional_pnl > total_pnl:
                    total_pnl += additional_pnl
                    extended_hours += 1
                else:
                    break
            
            run_hours = min_run_hours + extended_hours
            df.iloc[i:i+run_hours, df.columns.get_loc('run_decision')] = True
            df.iloc[i:i+run_hours, df.columns.get_loc('pnl')] = df.iloc[i:i+run_hours]['potential_pnl']
            i += run_hours + min_down_hours  # Skip the downtime after running
        else:
            i += 1

    return df

# Example DataFrame for demonstration
data = {
    'time': pd.date_range(start='2022-01-01', periods=24, freq='H'),
    'power_price': np.random.uniform(50, 150, 24),  # Simulated power prices
    'gas_price': np.random.uniform(3, 10, 24),      # Simulated gas prices
}
df = pd.DataFrame(data)
df.set_index('time', inplace=True)

# Call the function
optimized_df = calculate_optimal_pnl(df)

# Display the optimized DataFrame
print(optimized_df[['power_price', 'gas_price', 'potential_pnl', 'run_decision', 'pnl']])
